# Census Data for San Antonio Districts

In [1]:
import pandas as pd
import numpy as np
import re

from ccd_functions import read_distict, cal_counts, cal_district_numbers

## District Data

This sheet tells which district each census tract aligns with and what percentage of the district is included. District 0 are census tracts outside of San Antonio.

In [2]:
dist = read_distict()
dist.head()

,council,tract,percent
0,1.0,192000,0.221309
1,1.0,191900,0.000049
2,1.0,192300,0.185373
3,1.0,192100,0.572719
4,1.0,191304,0.786238


## Census Data

This is all the data pulled from the census. 

In [3]:
filename = 'ACSDP5Y2021.DP04-Data.csv'
filename2 = 'ACSDP5Y2021.DP04-Column-Metadata.csv'

In [4]:
dataall = pd.read_csv(filename)
dataall.columns = dataall.columns.str.lower()

dataall = dataall.iloc[1:].reset_index(drop=True)
dataall = dataall.replace('(X)', np.nan).replace('-',np.nan).replace('\*+',np.nan,regex=True)
# dataall = dataall.dropna(axis=1, how='all')

dataall.geo_id = dataall.geo_id.str[-6:]
dataall.geo_id = dataall.geo_id.astype(str)
dataall = dataall.drop(columns='name')

to_fix = []
for col in dataall.columns:
    bad_num = dataall[col].astype(str).str.contains('+',regex=False).sum()
    if bad_num > 0:
        to_fix.append(col)
for col in to_fix:
    dataall[col] = dataall[col].astype(str)\
    .str.replace('+','',regex=False)\
    .str.replace(',','').astype(float)

dataall.head()

,geo_id,dp04_0001e,dp04_0001ea,dp04_0001m,dp04_0001ma,dp04_0002e,dp04_0002m,dp04_0002ma,dp04_0002ea,dp04_0003e,...,dp04_0141pea,dp04_0142pe,dp04_0142pm,dp04_0142pma,dp04_0142pea,dp04_0143pe,dp04_0143pm,dp04_0143pma,dp04_0143pea,unnamed: 1146
0,110100,2958,NaN,272,NaN,1895,286,NaN,NaN,1063,...,NaN,31.1,11.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,110300,1888,NaN,265,NaN,1393,251,NaN,NaN,495,...,NaN,40.4,14.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,110500,1025,NaN,117,NaN,921,133,NaN,NaN,104,...,NaN,41.4,11.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,110600,1637,NaN,255,NaN,1551,269,NaN,NaN,86,...,NaN,51.9,13.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,110700,661,NaN,116,NaN,513,104,NaN,NaN,148,...,NaN,43.2,18.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Labels

These are the labels for the census data.

In [5]:
labels = pd.read_csv(filename2)
labels.columns = labels.columns.str.lower().str.replace('\W+','_', regex=True)
labels.column_name = labels.column_name.str.lower()

labels = labels [labels.label.str.startswith('Estimate')]
labels = labels [~labels.label.str.contains('ratio')]

groups = labels.label.str.split('!', expand=True)[2].unique()
groups

array(['HOUSING OCCUPANCY', 'UNITS IN STRUCTURE', 'YEAR STRUCTURE BUILT',
       'ROOMS', 'BEDROOMS', 'HOUSING TENURE',
       'YEAR HOUSEHOLDER MOVED INTO UNIT', 'VEHICLES AVAILABLE',
       'HOUSE HEATING FUEL', 'SELECTED CHARACTERISTICS',
       'OCCUPANTS PER ROOM', 'VALUE', 'MORTGAGE STATUS',
       'SELECTED MONTHLY OWNER COSTS (SMOC)',
       'SELECTED MONTHLY OWNER COSTS AS A PERCENTAGE OF HOUSEHOLD INCOME (SMOCAPI)',
       'GROSS RENT',
       'GROSS RENT AS A PERCENTAGE OF HOUSEHOLD INCOME (GRAPI)'],
      dtype=object)

## Now combine and calculate! 

### Housing units

In [6]:
grouping = groups[0]
print(grouping)
print()

current_labels = labels [labels.label.str.contains(grouping)]
labels_check = current_labels.column_name.str[:-1]

for titles in current_labels.label:
    print(titles)
    
final1 = cal_district_numbers(dist, dataall, labels, labels_check, 'DP04_0001E')
final1

HOUSING OCCUPANCY

Estimate!!HOUSING OCCUPANCY!!Total housing units
Estimate!!HOUSING OCCUPANCY!!Total housing units!!Occupied housing units
Estimate!!HOUSING OCCUPANCY!!Total housing units!!Vacant housing units
Estimate!!HOUSING OCCUPANCY!!Total housing units!!Homeowner vacancy rate
Estimate!!HOUSING OCCUPANCY!!Total housing units!!Rental vacancy rate


,0,1,2,3,4,5,6,7,8,9,10
Total housing units counts,169004.87,62543.29,51190.97,53805.84,43728.79,51617.79,46648.37,63340.62,68283.71,54064.33,58664.41
Total housing units counts percent,0.23,0.09,0.07,0.07,0.06,0.07,0.06,0.09,0.09,0.07,0.08
Total housing units moe,2357.91,1418.24,1164.17,1189.01,939.73,1208.81,1182.50,1426.58,1404.72,1189.99,1151.38
Total housing units moa percent,0.01,0.01,0.01,0.01,0.01,0.01,0.02,0.01,0.01,0.01,0.01
Occupied housing units counts,159118.98,55272.50,45724.55,49089.50,40906.32,46251.65,44545.92,58525.95,61494.52,50336.81,55171.31
Occupied housing units counts percent,0.22,0.08,0.06,0.07,0.06,0.06,0.06,0.08,0.09,0.07,0.08
Occupied housing units moe,2369.98,1461.04,1185.53,1228.99,990.87,1219.99,1224.41,1480.28,1481.76,1214.78,1165.15
Occupied housing units moa percent,0.01,0.02,0.02,0.02,0.01,0.02,0.02,0.02,0.01,0.01,0.01
Vacant housing units counts,9885.89,7270.80,5466.43,4716.34,2822.47,5366.14,2102.45,4814.67,6789.20,3727.52,3493.10
Vacant housing units counts percent,0.01,0.01,0.01,0.01,0.00,0.01,0.00,0.01,0.01,0.01,0.00


### Units in structure

In [7]:
grouping = groups[1]
print(grouping)
print()

current_labels = labels [labels.label.str.contains(grouping)]
labels_check = current_labels.column_name.str[:-1]

for titles in current_labels.label:
    print(titles)
    
final2 = cal_district_numbers(dist, dataall, labels, labels_check, 'DP04_0006E')    
final2

UNITS IN STRUCTURE

Estimate!!UNITS IN STRUCTURE!!Total housing units
Estimate!!UNITS IN STRUCTURE!!Total housing units!!1-unit, detached
Estimate!!UNITS IN STRUCTURE!!Total housing units!!1-unit, attached
Estimate!!UNITS IN STRUCTURE!!Total housing units!!2 units
Estimate!!UNITS IN STRUCTURE!!Total housing units!!3 or 4 units
Estimate!!UNITS IN STRUCTURE!!Total housing units!!5 to 9 units
Estimate!!UNITS IN STRUCTURE!!Total housing units!!10 to 19 units
Estimate!!UNITS IN STRUCTURE!!Total housing units!!20 or more units
Estimate!!UNITS IN STRUCTURE!!Total housing units!!Mobile home
Estimate!!UNITS IN STRUCTURE!!Total housing units!!Boat, RV, van, etc.


/Users/mists/Projects/sa2020/sa2020-ccd/ccd_functions.py:69: RuntimeWarning: divide by zero encountered in double_scalars
  error_perc = round(math.sqrt(sum((subset.tru_error / 1.645)**2)) / count, 2)


,0,1,2,3,4,5,6,7,8,9,10
Total housing units counts,169004.87,62543.29,51190.97,53805.84,43728.79,51617.79,46648.37,63340.62,68283.71,54064.33,58664.41
Total housing units counts percent,0.23,0.09,0.07,0.07,0.06,0.07,0.06,0.09,0.09,0.07,0.08
Total housing units moe,2357.91,1418.24,1164.17,1189.01,939.73,1208.81,1182.50,1426.58,1404.72,1189.99,1151.38
Total housing units moa percent,0.01,0.01,0.01,0.01,0.01,0.01,0.02,0.01,0.01,0.01,0.01
"1-unit, detached counts",131618.72,33696.99,34102.28,37570.66,33637.77,38946.29,33472.66,37702.75,24215.55,29795.43,39116.90
"1-unit, detached counts percent",0.18,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.03,0.04,0.05
"1-unit, detached moe",2269.83,1101.55,969.23,1094.65,902.79,1109.29,1098.90,1170.83,898.19,844.13,1002.03
"1-unit, detached moa percent",0.01,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02
"1-unit, attached counts",3534.33,1550.72,1242.96,786.15,820.01,1021.76,1006.61,2553.95,1672.73,1038.80,1555.97
"1-unit, attached counts percent",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


### Year Built

In [8]:
grouping = groups[2]
print(grouping)
print()

current_labels = labels [labels.label.str.contains(grouping)]
labels_check = current_labels.column_name.str[:-1]

for titles in current_labels.label:
    print(titles)
    
final3 = cal_district_numbers(dist, dataall, labels, labels_check, 'DP04_0016E')
final3

YEAR STRUCTURE BUILT

Estimate!!YEAR STRUCTURE BUILT!!Total housing units
Estimate!!YEAR STRUCTURE BUILT!!Total housing units!!Built 2020 or later
Estimate!!YEAR STRUCTURE BUILT!!Total housing units!!Built 2010 to 2019
Estimate!!YEAR STRUCTURE BUILT!!Total housing units!!Built 2000 to 2009
Estimate!!YEAR STRUCTURE BUILT!!Total housing units!!Built 1990 to 1999
Estimate!!YEAR STRUCTURE BUILT!!Total housing units!!Built 1980 to 1989
Estimate!!YEAR STRUCTURE BUILT!!Total housing units!!Built 1970 to 1979
Estimate!!YEAR STRUCTURE BUILT!!Total housing units!!Built 1960 to 1969
Estimate!!YEAR STRUCTURE BUILT!!Total housing units!!Built 1950 to 1959
Estimate!!YEAR STRUCTURE BUILT!!Total housing units!!Built 1940 to 1949
Estimate!!YEAR STRUCTURE BUILT!!Total housing units!!Built 1939 or earlier


,0,1,2,3,4,5,6,7,8,9,10
Total housing units counts,169004.87,62543.29,51190.97,53805.84,43728.79,51617.79,46648.37,63340.62,68283.71,54064.33,58664.41
Total housing units counts percent,0.23,0.09,0.07,0.07,0.06,0.07,0.06,0.09,0.09,0.07,0.08
Total housing units moe,2357.91,1418.24,1164.17,1189.01,939.73,1208.81,1182.50,1426.58,1404.72,1189.99,1151.38
Total housing units moa percent,0.01,0.01,0.01,0.01,0.01,0.01,0.02,0.01,0.01,0.01,0.01
Built 2020 or later counts,1655.13,23.09,349.68,97.46,119.65,42.30,33.97,149.81,77.69,133.72,44.51
Built 2020 or later counts percent,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Built 2020 or later moe,356.93,91.04,163.25,111.16,104.81,92.12,82.25,126.93,100.64,126.82,94.50
Built 2020 or later moa percent,0.13,2.40,0.28,0.69,0.53,1.32,1.47,0.52,0.79,0.58,1.29
Built 2010 to 2019 counts,47875.71,2602.85,6475.07,5123.14,4047.54,1945.70,4876.31,4563.16,13717.77,5730.78,4650.95
Built 2010 to 2019 counts percent,0.07,0.00,0.01,0.01,0.01,0.00,0.01,0.01,0.02,0.01,0.01


### Rooms

In [9]:
grouping = groups[3]
print(grouping)
print()

current_labels = labels [labels.label.str.contains(grouping)]
current_labels = current_labels.iloc[:-7]
labels_check = current_labels.column_name.str[:-1]

for titles in current_labels.label:
    print(titles)
    
final4 = cal_district_numbers(dist, dataall, labels, labels_check, 'DP04_0027E')
final4

ROOMS

Estimate!!ROOMS!!Total housing units
Estimate!!ROOMS!!Total housing units!!1 room
Estimate!!ROOMS!!Total housing units!!2 rooms
Estimate!!ROOMS!!Total housing units!!3 rooms
Estimate!!ROOMS!!Total housing units!!4 rooms
Estimate!!ROOMS!!Total housing units!!5 rooms
Estimate!!ROOMS!!Total housing units!!6 rooms
Estimate!!ROOMS!!Total housing units!!7 rooms
Estimate!!ROOMS!!Total housing units!!8 rooms
Estimate!!ROOMS!!Total housing units!!9 rooms or more
Estimate!!ROOMS!!Total housing units!!Median rooms


,0,1,2,3,4,5,6,7,8,9,10
Total housing units counts,169004.87,62543.29,51190.97,53805.84,43728.79,51617.79,46648.37,63340.62,68283.71,54064.33,58664.41
Total housing units counts percent,0.23,0.09,0.07,0.07,0.06,0.07,0.06,0.09,0.09,0.07,0.08
Total housing units moe,2357.91,1418.24,1164.17,1189.01,939.73,1208.81,1182.50,1426.58,1404.72,1189.99,1151.38
Total housing units moa percent,0.01,0.01,0.01,0.01,0.01,0.01,0.02,0.01,0.01,0.01,0.01
1 room counts,2401.79,3309.27,1425.98,1685.30,799.49,1693.67,976.92,1900.62,3382.44,1477.60,1506.90
1 room counts percent,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1 room moe,438.34,524.22,282.41,325.85,253.55,316.49,255.00,373.37,501.61,450.98,378.91
1 room moa percent,0.11,0.10,0.12,0.12,0.19,0.11,0.16,0.12,0.09,0.19,0.15
2 rooms counts,3657.02,2848.18,2007.45,2367.37,1106.52,1325.84,1157.77,2889.45,6063.21,2020.73,2166.45
2 rooms counts percent,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.01,0.00,0.00


### Bedrooms

In [10]:
grouping = groups[4]
print(grouping)
print()

current_labels = labels [labels.label.str.contains(grouping)]
labels_check = current_labels.column_name.str[:-1]

for titles in current_labels.label:
    print(titles)
    
final5 = cal_district_numbers(dist, dataall, labels, labels_check, 'DP04_0038E')
final5

BEDROOMS

Estimate!!BEDROOMS!!Total housing units
Estimate!!BEDROOMS!!Total housing units!!No bedroom
Estimate!!BEDROOMS!!Total housing units!!1 bedroom
Estimate!!BEDROOMS!!Total housing units!!2 bedrooms
Estimate!!BEDROOMS!!Total housing units!!3 bedrooms
Estimate!!BEDROOMS!!Total housing units!!4 bedrooms
Estimate!!BEDROOMS!!Total housing units!!5 or more bedrooms


,0,1,2,3,4,5,6,7,8,9,10
Total housing units counts,169004.87,62543.29,51190.97,53805.84,43728.79,51617.79,46648.37,63340.62,68283.71,54064.33,58664.41
Total housing units counts percent,0.23,0.09,0.07,0.07,0.06,0.07,0.06,0.09,0.09,0.07,0.08
Total housing units moe,2357.91,1418.24,1164.17,1189.01,939.73,1208.81,1182.50,1426.58,1404.72,1189.99,1151.38
Total housing units moa percent,0.01,0.01,0.01,0.01,0.01,0.01,0.02,0.01,0.01,0.01,0.01
No bedroom counts,2636.08,3571.58,1489.31,1745.02,908.25,1763.41,1046.54,2070.87,3455.44,1601.60,1565.90
No bedroom counts percent,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
No bedroom moe,456.13,530.27,283.61,327.09,259.38,320.51,261.31,387.99,503.03,460.26,381.93
No bedroom moa percent,0.11,0.09,0.12,0.11,0.17,0.11,0.15,0.11,0.09,0.17,0.15
1 bedroom counts,10208.18,14033.00,7023.51,7214.64,3514.45,7428.20,5095.68,10792.82,18310.05,11095.48,7371.98
1 bedroom counts percent,0.01,0.02,0.01,0.01,0.00,0.01,0.01,0.01,0.03,0.02,0.01


### Tenure

In [11]:
grouping = groups[5]
print(grouping)
print()

current_labels = labels [labels.label.str.contains(grouping)]
labels_check = current_labels.column_name.str[:-1]

for titles in current_labels.label:
    print(titles)
    
final6 = cal_district_numbers(dist, dataall, labels, labels_check, 'DP04_0045E')
final6

HOUSING TENURE

Estimate!!HOUSING TENURE!!Occupied housing units
Estimate!!HOUSING TENURE!!Occupied housing units!!Owner-occupied
Estimate!!HOUSING TENURE!!Occupied housing units!!Renter-occupied
Estimate!!HOUSING TENURE!!Occupied housing units!!Average household size of owner-occupied unit
Estimate!!HOUSING TENURE!!Occupied housing units!!Average household size of renter-occupied unit


,0,1,2,3,4,5,6,7,8,9,10
Occupied housing units counts,159118.98,55272.50,45724.55,49089.50,40906.32,46251.65,44545.92,58525.95,61494.52,50336.81,55171.31
Occupied housing units counts percent,0.24,0.08,0.07,0.07,0.06,0.07,0.07,0.09,0.09,0.08,0.08
Occupied housing units moe,2369.98,1461.04,1185.53,1228.99,990.87,1219.99,1224.41,1480.28,1481.76,1214.78,1165.15
Occupied housing units moa percent,0.01,0.02,0.02,0.02,0.01,0.02,0.02,0.02,0.01,0.01,0.01
Owner-occupied counts,119567.31,25580.80,23105.91,29057.00,25757.26,25986.20,26942.04,31817.28,22097.90,27709.99,34380.31
Owner-occupied counts percent,0.18,0.04,0.03,0.04,0.04,0.04,0.04,0.05,0.03,0.04,0.05
Owner-occupied moe,2265.33,1018.17,915.60,1064.94,915.89,998.96,1054.39,1198.82,869.85,788.20,1089.97
Owner-occupied moa percent,0.01,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02
Renter-occupied counts,39551.66,29691.70,22618.64,20032.50,15149.06,20265.45,17603.88,26708.67,39396.61,22626.82,20791.00
Renter-occupied counts percent,0.06,0.04,0.03,0.03,0.02,0.03,0.03,0.04,0.06,0.03,0.03


### YEAR HOUSEHOLDER MOVED INTO UNIT

In [12]:
grouping = groups[6]
print(grouping)
print()

current_labels = labels [labels.label.str.contains(grouping)]
labels_check = current_labels.column_name.str[:-1]

for titles in current_labels.label:
    print(titles)
    
final7 = cal_district_numbers(dist, dataall, labels, labels_check, 'DP04_0050E')
final7

YEAR HOUSEHOLDER MOVED INTO UNIT

Estimate!!YEAR HOUSEHOLDER MOVED INTO UNIT!!Occupied housing units
Estimate!!YEAR HOUSEHOLDER MOVED INTO UNIT!!Occupied housing units!!Moved in 2019 or later
Estimate!!YEAR HOUSEHOLDER MOVED INTO UNIT!!Occupied housing units!!Moved in 2015 to 2018
Estimate!!YEAR HOUSEHOLDER MOVED INTO UNIT!!Occupied housing units!!Moved in 2010 to 2014
Estimate!!YEAR HOUSEHOLDER MOVED INTO UNIT!!Occupied housing units!!Moved in 2000 to 2009
Estimate!!YEAR HOUSEHOLDER MOVED INTO UNIT!!Occupied housing units!!Moved in 1990 to 1999
Estimate!!YEAR HOUSEHOLDER MOVED INTO UNIT!!Occupied housing units!!Moved in 1989 and earlier


,0,1,2,3,4,5,6,7,8,9,10
Occupied housing units counts,159118.98,55272.50,45724.55,49089.50,40906.32,46251.65,44545.92,58525.95,61494.52,50336.81,55171.31
Occupied housing units counts percent,0.24,0.08,0.07,0.07,0.06,0.07,0.07,0.09,0.09,0.08,0.08
Occupied housing units moe,2369.98,1461.04,1185.53,1228.99,990.87,1219.99,1224.41,1480.28,1481.76,1214.78,1165.15
Occupied housing units moa percent,0.01,0.02,0.02,0.02,0.01,0.02,0.02,0.02,0.01,0.01,0.01
Moved in 2019 or later counts,19519.22,7477.29,6618.24,5137.85,4007.88,4154.57,5695.53,8685.04,13975.14,7921.48,6812.75
Moved in 2019 or later counts percent,0.03,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.02,0.01,0.01
Moved in 2019 or later moe,1377.11,785.36,742.28,678.82,498.06,617.95,775.26,890.70,1140.88,926.47,799.26
Moved in 2019 or later moa percent,0.04,0.06,0.07,0.08,0.08,0.09,0.08,0.06,0.05,0.07,0.07
Moved in 2015 to 2018 counts,55484.72,19490.92,15567.06,14134.96,10716.73,10895.15,13174.10,18416.15,27247.36,19170.28,19064.58
Moved in 2015 to 2018 counts percent,0.08,0.03,0.02,0.02,0.02,0.02,0.02,0.03,0.04,0.03,0.03


### VEHICLES AVAILABLE

In [13]:
grouping = groups[7]
print(grouping)
print()

current_labels = labels [labels.label.str.contains(grouping)]
labels_check = current_labels.column_name.str[:-1]

for titles in current_labels.label:
    print(titles)
    
final8 = cal_district_numbers(dist, dataall, labels, labels_check, 'DP04_0057E')
final8

VEHICLES AVAILABLE

Estimate!!VEHICLES AVAILABLE!!Occupied housing units
Estimate!!VEHICLES AVAILABLE!!Occupied housing units!!No vehicles available
Estimate!!VEHICLES AVAILABLE!!Occupied housing units!!1 vehicle available
Estimate!!VEHICLES AVAILABLE!!Occupied housing units!!2 vehicles available
Estimate!!VEHICLES AVAILABLE!!Occupied housing units!!3 or more vehicles available


,0,1,2,3,4,5,6,7,8,9,10
Occupied housing units counts,159118.98,55272.50,45724.55,49089.50,40906.32,46251.65,44545.92,58525.95,61494.52,50336.81,55171.31
Occupied housing units counts percent,0.24,0.08,0.07,0.07,0.06,0.07,0.07,0.09,0.09,0.08,0.08
Occupied housing units moe,2369.98,1461.04,1185.53,1228.99,990.87,1219.99,1224.41,1480.28,1481.76,1214.78,1165.15
Occupied housing units moa percent,0.01,0.02,0.02,0.02,0.01,0.02,0.02,0.02,0.01,0.01,0.01
No vehicles available counts,4255.31,6084.21,4347.37,5916.90,2252.12,6892.72,1808.74,4743.01,3177.75,2127.43,2871.43
No vehicles available counts percent,0.01,0.01,0.01,0.01,0.00,0.01,0.00,0.01,0.00,0.00,0.00
No vehicles available moe,485.70,629.67,456.78,705.97,356.56,710.30,384.89,595.31,508.08,410.07,525.44
No vehicles available moa percent,0.07,0.06,0.06,0.07,0.10,0.06,0.13,0.08,0.10,0.12,0.11
1 vehicle available counts,41999.63,22731.13,17286.80,16571.93,12708.82,16671.21,15136.91,23027.49,27287.35,18494.44,19184.29
1 vehicle available counts percent,0.06,0.03,0.03,0.02,0.02,0.03,0.02,0.03,0.04,0.03,0.03


### HOUSE HEATING FUEL

In [14]:
grouping = groups[8]
print(grouping)
print()

current_labels = labels [labels.label.str.contains(grouping)]
labels_check = current_labels.column_name.str[:-1]

for titles in current_labels.label:
    print(titles)
    
final9 = cal_district_numbers(dist, dataall, labels, labels_check, 'DP04_0062E')
final9

HOUSE HEATING FUEL

Estimate!!HOUSE HEATING FUEL!!Occupied housing units
Estimate!!HOUSE HEATING FUEL!!Occupied housing units!!Utility gas
Estimate!!HOUSE HEATING FUEL!!Occupied housing units!!Bottled, tank, or LP gas
Estimate!!HOUSE HEATING FUEL!!Occupied housing units!!Electricity
Estimate!!HOUSE HEATING FUEL!!Occupied housing units!!Fuel oil, kerosene, etc.
Estimate!!HOUSE HEATING FUEL!!Occupied housing units!!Coal or coke
Estimate!!HOUSE HEATING FUEL!!Occupied housing units!!Wood
Estimate!!HOUSE HEATING FUEL!!Occupied housing units!!Solar energy
Estimate!!HOUSE HEATING FUEL!!Occupied housing units!!Other fuel
Estimate!!HOUSE HEATING FUEL!!Occupied housing units!!No fuel used


,0,1,2,3,4,5,6,7,8,9,10
Occupied housing units counts,159118.98,55272.50,45724.55,49089.50,40906.32,46251.65,44545.92,58525.95,61494.52,50336.81,55171.31
Occupied housing units counts percent,0.24,0.08,0.07,0.07,0.06,0.07,0.07,0.09,0.09,0.08,0.08
Occupied housing units moe,2369.98,1461.04,1185.53,1228.99,990.87,1219.99,1224.41,1480.28,1481.76,1214.78,1165.15
Occupied housing units moa percent,0.01,0.02,0.02,0.02,0.01,0.02,0.02,0.02,0.01,0.01,0.01
Utility gas counts,42154.41,18549.44,13427.83,18286.78,11898.37,19677.95,11913.56,18382.15,13396.32,13367.52,16440.67
Utility gas counts percent,0.06,0.03,0.02,0.03,0.02,0.03,0.02,0.03,0.02,0.02,0.02
Utility gas moe,1430.10,830.04,764.95,971.76,783.15,972.18,845.96,1023.15,803.68,656.14,901.21
Utility gas moa percent,0.02,0.03,0.03,0.03,0.04,0.03,0.04,0.03,0.04,0.03,0.03
"Bottled, tank, or LP gas counts",3280.03,265.10,332.84,447.91,372.13,237.44,150.80,208.27,425.48,572.95,320.05
"Bottled, tank, or LP gas counts percent",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


### SELECTED CHARACTERISTICS

In [15]:
grouping = groups[9]
print(grouping)
print()

current_labels = labels [labels.label.str.contains(grouping)]
labels_check = current_labels.column_name.str[:-1]

for titles in current_labels.label:
    print(titles)
    
final10 = cal_district_numbers(dist, dataall, labels, labels_check, 'DP04_0072E')
final10

SELECTED CHARACTERISTICS

Estimate!!SELECTED CHARACTERISTICS!!Occupied housing units
Estimate!!SELECTED CHARACTERISTICS!!Occupied housing units!!Lacking complete plumbing facilities
Estimate!!SELECTED CHARACTERISTICS!!Occupied housing units!!Lacking complete kitchen facilities
Estimate!!SELECTED CHARACTERISTICS!!Occupied housing units!!No telephone service available


,0,1,2,3,4,5,6,7,8,9,10
Occupied housing units counts,159118.98,55272.50,45724.55,49089.50,40906.32,46251.65,44545.92,58525.95,61494.52,50336.81,55171.31
Occupied housing units counts percent,0.24,0.08,0.07,0.07,0.06,0.07,0.07,0.09,0.09,0.08,0.08
Occupied housing units moe,2369.98,1461.04,1185.53,1228.99,990.87,1219.99,1224.41,1480.28,1481.76,1214.78,1165.15
Occupied housing units moa percent,0.01,0.02,0.02,0.02,0.01,0.02,0.02,0.02,0.01,0.01,0.01
Lacking complete plumbing facilities counts,903.79,302.31,439.92,300.45,232.63,337.58,164.20,274.38,125.16,69.24,148.35
Lacking complete plumbing facilities counts percent,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Lacking complete plumbing facilities moe,380.94,169.71,181.58,153.66,132.06,131.27,110.15,163.59,155.06,96.90,123.91
Lacking complete plumbing facilities moa percent,0.26,0.34,0.25,0.31,0.35,0.24,0.41,0.36,0.75,0.85,0.51
Lacking complete kitchen facilities counts,595.90,539.09,337.52,445.38,245.25,471.77,180.29,413.40,505.46,190.57,561.39
Lacking complete kitchen facilities counts percent,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


### OCCUPANTS PER ROOM

In [16]:
grouping = groups[10]
print(grouping)
print()

current_labels = labels [labels.label.str.contains(grouping)]
labels_check = current_labels.column_name.str[:-1]

for titles in current_labels.label:
    print(titles)
    
final11 = cal_district_numbers(dist, dataall, labels, labels_check, 'DP04_0076E')
final11

OCCUPANTS PER ROOM

Estimate!!OCCUPANTS PER ROOM!!Occupied housing units
Estimate!!OCCUPANTS PER ROOM!!Occupied housing units!!1.00 or less
Estimate!!OCCUPANTS PER ROOM!!Occupied housing units!!1.01 to 1.50
Estimate!!OCCUPANTS PER ROOM!!Occupied housing units!!1.51 or more


,0,1,2,3,4,5,6,7,8,9,10
Occupied housing units counts,159118.98,55272.50,45724.55,49089.50,40906.32,46251.65,44545.92,58525.95,61494.52,50336.81,55171.31
Occupied housing units counts percent,0.24,0.08,0.07,0.07,0.06,0.07,0.07,0.09,0.09,0.08,0.08
Occupied housing units moe,2369.98,1461.04,1185.53,1228.99,990.87,1219.99,1224.41,1480.28,1481.76,1214.78,1165.15
Occupied housing units moa percent,0.01,0.02,0.02,0.02,0.01,0.02,0.02,0.02,0.01,0.01,0.01
1.00 or less counts,153813.85,52617.80,43493.68,45924.26,37676.35,42079.32,42879.73,56243.19,59016.45,48977.84,53487.54
1.00 or less counts percent,0.23,0.08,0.07,0.07,0.06,0.06,0.06,0.08,0.09,0.07,0.08
1.00 or less moe,2397.77,1475.26,1192.17,1250.45,1014.70,1205.51,1228.74,1453.97,1475.48,1188.60,1183.31
1.00 or less moa percent,0.01,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.01,0.01
1.01 to 1.50 counts,3289.46,1537.88,1483.65,2450.92,2474.84,2968.76,1245.78,1427.45,1541.49,826.51,1071.26
1.01 to 1.50 counts percent,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


### VALUE

In [17]:
grouping = groups[11]
print(grouping)
print()

current_labels = labels [labels.label.str.contains(grouping)]
labels_check = current_labels.column_name.str[:-1]

for titles in current_labels.label:
    print(titles)
    
final12 = cal_district_numbers(dist, dataall, labels, labels_check, 'DP04_0080E')
final12

VALUE

Estimate!!VALUE!!Owner-occupied units
Estimate!!VALUE!!Owner-occupied units!!Less than $50,000
Estimate!!VALUE!!Owner-occupied units!!$50,000 to $99,999
Estimate!!VALUE!!Owner-occupied units!!$100,000 to $149,999
Estimate!!VALUE!!Owner-occupied units!!$150,000 to $199,999
Estimate!!VALUE!!Owner-occupied units!!$200,000 to $299,999
Estimate!!VALUE!!Owner-occupied units!!$300,000 to $499,999
Estimate!!VALUE!!Owner-occupied units!!$500,000 to $999,999
Estimate!!VALUE!!Owner-occupied units!!$1,000,000 or more
Estimate!!VALUE!!Owner-occupied units!!Median (dollars)


,0,1,2,3,4,5,6,7,8,9,10
Owner-occupied units counts,119567.31,25580.80,23105.91,29057.00,25757.26,25986.20,26942.04,31817.28,22097.90,27709.99,34380.31
Owner-occupied units counts percent,0.31,0.07,0.06,0.07,0.07,0.07,0.07,0.08,0.06,0.07,0.09
Owner-occupied units moe,2265.33,1018.17,915.60,1064.94,915.89,998.96,1054.39,1198.82,869.85,788.20,1089.97
Owner-occupied units moa percent,0.01,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02
"Less than $50,000 counts",4343.21,997.04,2381.74,1971.39,1965.49,4062.83,1165.12,855.77,413.35,279.12,856.94
"Less than $50,000 counts percent",0.01,0.00,0.01,0.01,0.01,0.01,0.00,0.00,0.00,0.00,0.00
"Less than $50,000 moe",466.48,226.43,334.40,296.49,398.67,397.60,240.97,226.67,185.16,120.06,216.77
"Less than $50,000 moa percent",0.07,0.14,0.09,0.09,0.12,0.06,0.13,0.16,0.27,0.26,0.15
"$50,000 to $99,999 counts",8174.74,3736.79,6118.91,11200.15,7727.94,13884.83,3236.17,3158.86,723.90,415.89,1567.81
"$50,000 to $99,999 counts percent",0.02,0.01,0.02,0.03,0.02,0.04,0.01,0.01,0.00,0.00,0.00


### MORTGAGE STATUS

In [18]:
grouping = groups[12]
print(grouping)
print()

current_labels = labels [labels.label.str.contains(grouping)]
labels_check = current_labels.column_name.str[:-1]

for titles in current_labels.label:
    print(titles)
    
final13 = cal_district_numbers(dist, dataall, labels, labels_check, 'DP04_0090E')
final13

MORTGAGE STATUS

Estimate!!MORTGAGE STATUS!!Owner-occupied units
Estimate!!MORTGAGE STATUS!!Owner-occupied units!!Housing units with a mortgage
Estimate!!MORTGAGE STATUS!!Owner-occupied units!!Housing units without a mortgage


,0,1,2,3,4,5,6,7,8,9,10
Owner-occupied units counts,119567.31,25580.80,23105.91,29057.00,25757.26,25986.20,26942.04,31817.28,22097.90,27709.99,34380.31
Owner-occupied units counts percent,0.31,0.07,0.06,0.07,0.07,0.07,0.07,0.08,0.06,0.07,0.09
Owner-occupied units moe,2265.33,1018.17,915.60,1064.94,915.89,998.96,1054.39,1198.82,869.85,788.20,1089.97
Owner-occupied units moa percent,0.01,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02
Housing units with a mortgage counts,82995.82,13183.01,12747.98,14451.82,14371.86,8464.36,16870.36,19191.94,14489.56,18356.64,22147.64
Housing units with a mortgage counts percent,0.21,0.03,0.03,0.04,0.04,0.02,0.04,0.05,0.04,0.05,0.06
Housing units with a mortgage moe,2149.01,769.15,785.61,837.18,726.24,625.91,895.28,1061.63,781.64,739.97,967.48
Housing units with a mortgage moa percent,0.02,0.04,0.04,0.04,0.03,0.04,0.03,0.03,0.03,0.02,0.03
Housing units without a mortgage counts,36571.50,12397.79,10357.92,14605.18,11385.40,17521.84,10071.67,12625.34,7608.34,9353.35,12232.67
Housing units without a mortgage counts percent,0.09,0.03,0.03,0.04,0.03,0.04,0.03,0.03,0.02,0.02,0.03


### SELECTED MONTHLY OWNER COSTS

In [19]:
grouping = groups[13]
grouping = grouping[:-7]
print(grouping)
print()

current_labels = labels [labels.label.str.contains(grouping)]
current_labels = current_labels.iloc[:-16]
labels_check = current_labels.column_name.str[:-1]

for titles in current_labels.label:
    print(titles)
    
final14 = cal_district_numbers(dist, dataall, labels, labels_check, 'DP04_0093E')
final14

SELECTED MONTHLY OWNER COSTS

Estimate!!SELECTED MONTHLY OWNER COSTS (SMOC)!!Housing units with a mortgage
Estimate!!SELECTED MONTHLY OWNER COSTS (SMOC)!!Housing units with a mortgage!!Less than $500
Estimate!!SELECTED MONTHLY OWNER COSTS (SMOC)!!Housing units with a mortgage!!$500 to $999
Estimate!!SELECTED MONTHLY OWNER COSTS (SMOC)!!Housing units with a mortgage!!$1,000 to $1,499
Estimate!!SELECTED MONTHLY OWNER COSTS (SMOC)!!Housing units with a mortgage!!$1,500 to $1,999
Estimate!!SELECTED MONTHLY OWNER COSTS (SMOC)!!Housing units with a mortgage!!$2,000 to $2,499
Estimate!!SELECTED MONTHLY OWNER COSTS (SMOC)!!Housing units with a mortgage!!$2,500 to $2,999
Estimate!!SELECTED MONTHLY OWNER COSTS (SMOC)!!Housing units with a mortgage!!$3,000 or more
Estimate!!SELECTED MONTHLY OWNER COSTS (SMOC)!!Housing units with a mortgage!!Median (dollars)
Estimate!!SELECTED MONTHLY OWNER COSTS (SMOC)!!Housing units without a mortgage
Estimate!!SELECTED MONTHLY OWNER COSTS (SMOC)!!Housing units 

,0,1,2,3,4,5,6,7,8,9,10
Housing units with a mortgage counts,82995.82,13183.01,12747.98,14451.82,14371.86,8464.36,16870.36,19191.94,14489.56,18356.64,22147.64
Housing units with a mortgage counts percent,0.35,0.06,0.05,0.06,0.06,0.04,0.07,0.08,0.06,0.08,0.09
Housing units with a mortgage moe,2149.01,769.15,785.61,837.18,726.24,625.91,895.28,1061.63,781.64,739.97,967.48
Housing units with a mortgage moa percent,0.02,0.04,0.04,0.04,0.03,0.04,0.03,0.03,0.03,0.02,0.03
Less than $500 counts,560.93,139.80,100.79,139.36,159.94,118.25,127.67,225.06,158.69,9.22,187.29
...,...,...,...,...,...,...,...,...,...,...,...
"$1,000 or more moa percent",0.06,0.10,0.22,0.24,0.31,0.34,0.29,0.14,0.07,0.06,0.10
Median (dollars) counts,46238.71,17737.43,11819.57,12828.18,10023.26,12471.92,11365.20,17338.85,19727.14,21961.48,16955.27
Median (dollars) counts percent,0.20,0.07,0.05,0.05,0.04,0.05,0.05,0.07,0.08,0.09,0.07
Median (dollars) moe,1488.13,758.91,578.22,583.12,315.87,505.51,369.96,533.19,1424.09,710.48,608.72


### SELECTED MONTHLY OWNER COSTS AS A PERCENTAGE OF HOUSEHOLD INCOME with mortgage

In [20]:
grouping = groups[14]
grouping = grouping[:-10]
print(grouping)
print()

current_labels = labels [labels.label.str.contains(grouping)]
current_labels = current_labels.iloc[:-9]
labels_check = current_labels.column_name.str[:-1]

for titles in current_labels.label:
    print(titles)
    
final15 = cal_district_numbers(dist, dataall, labels, labels_check, 'DP04_0110E')
final15

SELECTED MONTHLY OWNER COSTS AS A PERCENTAGE OF HOUSEHOLD INCOME

Estimate!!SELECTED MONTHLY OWNER COSTS AS A PERCENTAGE OF HOUSEHOLD INCOME (SMOCAPI)!!Housing units with a mortgage (excluding units where SMOCAPI cannot be computed)
Estimate!!SELECTED MONTHLY OWNER COSTS AS A PERCENTAGE OF HOUSEHOLD INCOME (SMOCAPI)!!Housing units with a mortgage (excluding units where SMOCAPI cannot be computed)!!Less than 20.0 percent
Estimate!!SELECTED MONTHLY OWNER COSTS AS A PERCENTAGE OF HOUSEHOLD INCOME (SMOCAPI)!!Housing units with a mortgage (excluding units where SMOCAPI cannot be computed)!!20.0 to 24.9 percent
Estimate!!SELECTED MONTHLY OWNER COSTS AS A PERCENTAGE OF HOUSEHOLD INCOME (SMOCAPI)!!Housing units with a mortgage (excluding units where SMOCAPI cannot be computed)!!25.0 to 29.9 percent
Estimate!!SELECTED MONTHLY OWNER COSTS AS A PERCENTAGE OF HOUSEHOLD INCOME (SMOCAPI)!!Housing units with a mortgage (excluding units where SMOCAPI cannot be computed)!!30.0 to 34.9 percent
Estimate!

,0,1,2,3,4,5,6,7,8,9,10
Housing units with a mortgage (excluding units where SMOCAPI cannot be computed) counts,82595.99,13155.18,12634.73,14404.12,14245.63,8420.98,16814.85,18968.09,14350.04,18198.97,22049.41
Housing units with a mortgage (excluding units where SMOCAPI cannot be computed) counts percent,0.35,0.06,0.05,0.06,0.06,0.04,0.07,0.08,0.06,0.08,0.09
Housing units with a mortgage (excluding units where SMOCAPI cannot be computed) moe,2146.71,768.77,783.24,837.94,725.38,625.51,891.23,1057.31,771.48,737.61,968.31
Housing units with a mortgage (excluding units where SMOCAPI cannot be computed) moa percent,0.02,0.04,0.04,0.04,0.03,0.05,0.03,0.03,0.03,0.02,0.03
Less than 20.0 percent counts,37526.73,5569.52,5658.00,6110.47,5985.47,3576.46,8085.04,9506.39,6832.08,8363.86,9319.97
Less than 20.0 percent counts percent,0.16,0.02,0.02,0.03,0.03,0.02,0.03,0.04,0.03,0.04,0.04
Less than 20.0 percent moe,1527.62,539.46,626.58,592.17,534.65,409.07,740.35,781.27,587.46,564.73,696.39
Less than 20.0 percent moa percent,0.02,0.06,0.07,0.06,0.05,0.07,0.06,0.05,0.05,0.04,0.05
20.0 to 24.9 percent counts,14116.03,2053.29,1683.75,2109.82,2049.14,985.41,2593.41,2826.87,2337.39,2964.88,4349.01
20.0 to 24.9 percent counts percent,0.06,0.01,0.01,0.01,0.01,0.00,0.01,0.01,0.01,0.01,0.02


### SELECTED MONTHLY OWNER COSTS AS A PERCENTAGE OF HOUSEHOLD INCOME without mortgage

In [21]:
grouping = groups[14]
grouping = grouping[:-10]
print(grouping)
print()

current_labels = labels [labels.label.str.contains(grouping)]
current_labels = current_labels.iloc[7:]
labels_check = current_labels.column_name.str[:-1]

for titles in current_labels.label:
    print(titles)
    
final16 = cal_district_numbers(dist, dataall, labels, labels_check, 'DP04_0117E')
final16

SELECTED MONTHLY OWNER COSTS AS A PERCENTAGE OF HOUSEHOLD INCOME

Estimate!!SELECTED MONTHLY OWNER COSTS AS A PERCENTAGE OF HOUSEHOLD INCOME (SMOCAPI)!!Housing unit without a mortgage (excluding units where SMOCAPI cannot be computed)
Estimate!!SELECTED MONTHLY OWNER COSTS AS A PERCENTAGE OF HOUSEHOLD INCOME (SMOCAPI)!!Housing unit without a mortgage (excluding units where SMOCAPI cannot be computed)!!Less than 10.0 percent
Estimate!!SELECTED MONTHLY OWNER COSTS AS A PERCENTAGE OF HOUSEHOLD INCOME (SMOCAPI)!!Housing unit without a mortgage (excluding units where SMOCAPI cannot be computed)!!10.0 to 14.9 percent
Estimate!!SELECTED MONTHLY OWNER COSTS AS A PERCENTAGE OF HOUSEHOLD INCOME (SMOCAPI)!!Housing unit without a mortgage (excluding units where SMOCAPI cannot be computed)!!15.0 to 19.9 percent
Estimate!!SELECTED MONTHLY OWNER COSTS AS A PERCENTAGE OF HOUSEHOLD INCOME (SMOCAPI)!!Housing unit without a mortgage (excluding units where SMOCAPI cannot be computed)!!20.0 to 24.9 percent

,0,1,2,3,4,5,6,7,8,9,10
Housing unit without a mortgage (excluding units where SMOCAPI cannot be computed) counts,36210.15,12193.76,9997.45,14267.66,11238.92,17112.68,10025.64,12145.04,7476.38,9259.51,12057.81
Housing unit without a mortgage (excluding units where SMOCAPI cannot be computed) counts percent,0.24,0.08,0.07,0.09,0.07,0.11,0.07,0.08,0.05,0.06,0.08
Housing unit without a mortgage (excluding units where SMOCAPI cannot be computed) moe,1396.81,862.94,647.99,827.72,746.15,904.21,785.45,780.04,593.04,575.31,831.34
Housing unit without a mortgage (excluding units where SMOCAPI cannot be computed) moa percent,0.02,0.04,0.04,0.04,0.04,0.03,0.05,0.04,0.05,0.04,0.04
Less than 10.0 percent counts,17479.68,4663.29,4042.04,6264.79,5307.12,7023.84,5000.00,5238.07,3187.12,4202.26,5602.79
Less than 10.0 percent counts percent,0.12,0.03,0.03,0.04,0.03,0.05,0.03,0.03,0.02,0.03,0.04
Less than 10.0 percent moe,993.61,517.30,450.80,530.43,506.63,533.90,595.26,546.94,370.56,392.68,576.85
Less than 10.0 percent moa percent,0.03,0.07,0.07,0.05,0.06,0.05,0.07,0.06,0.07,0.06,0.06
10.0 to 14.9 percent counts,7721.54,2564.51,2115.56,3123.70,2481.49,3545.29,1671.79,2658.40,1364.76,1817.83,2651.14
10.0 to 14.9 percent counts percent,0.05,0.02,0.01,0.02,0.02,0.02,0.01,0.02,0.01,0.01,0.02


### GROSS RENT

In [22]:
grouping = groups[15]
print(grouping)
print()

current_labels = labels [labels.label.str.contains(grouping)]
current_labels = current_labels.iloc[:-8]
labels_check = current_labels.column_name.str[:-1]

for titles in current_labels.label:
    print(titles)
    
final17 = cal_district_numbers(dist, dataall, labels, labels_check, 'DP04_0126E')
final17

GROSS RENT

Estimate!!GROSS RENT!!Occupied units paying rent
Estimate!!GROSS RENT!!Occupied units paying rent!!Less than $500
Estimate!!GROSS RENT!!Occupied units paying rent!!$500 to $999
Estimate!!GROSS RENT!!Occupied units paying rent!!$1,000 to $1,499
Estimate!!GROSS RENT!!Occupied units paying rent!!$1,500 to $1,999
Estimate!!GROSS RENT!!Occupied units paying rent!!$2,000 to $2,499
Estimate!!GROSS RENT!!Occupied units paying rent!!$2,500 to $2,999
Estimate!!GROSS RENT!!Occupied units paying rent!!$3,000 or more
Estimate!!GROSS RENT!!Occupied units paying rent!!Median (dollars)
Estimate!!GROSS RENT!!Occupied units paying rent!!No rent paid


,0,1,2,3,4,5,6,7,8,9,10
Occupied units paying rent counts,37867.52,28594.35,21217.36,18526.34,14456.72,18448.12,16997.03,25847.94,38640.68,22164.22,19866.71
Occupied units paying rent counts percent,0.14,0.11,0.08,0.07,0.06,0.07,0.06,0.10,0.15,0.08,0.08
Occupied units paying rent moe,1492.97,1235.10,1087.92,1002.63,864.34,962.85,965.80,1124.48,1381.48,1067.16,923.22
Occupied units paying rent moa percent,0.02,0.03,0.03,0.03,0.04,0.03,0.03,0.03,0.02,0.03,0.03
Less than $500 counts,892.69,2327.11,2208.16,2325.08,749.64,3678.97,490.58,1907.56,406.91,147.76,428.54
Less than $500 counts percent,0.00,0.01,0.01,0.01,0.00,0.01,0.00,0.01,0.00,0.00,0.00
Less than $500 moe,236.22,326.40,340.47,389.45,231.23,441.64,177.75,336.98,190.03,102.93,172.35
Less than $500 moa percent,0.16,0.09,0.09,0.10,0.19,0.07,0.22,0.11,0.28,0.42,0.24
$500 to $999 counts,8614.70,12016.96,7647.64,8622.40,6424.68,9308.34,5427.88,9097.82,10119.89,5904.03,5728.66
$500 to $999 counts percent,0.03,0.05,0.03,0.03,0.02,0.04,0.02,0.03,0.04,0.02,0.02


### GROSS RENT AS A PERCENTAGE OF HOUSEHOLD INCOME

In [23]:
grouping = groups[16]
grouping = grouping[:-8]
print(grouping)
print()

current_labels = labels [labels.label.str.contains(grouping)]
labels_check = current_labels.column_name.str[:-1]

for titles in current_labels.label:
    print(titles)
    
final18 = cal_district_numbers(dist, dataall, labels, labels_check,'DP04_0136E')
final18

GROSS RENT AS A PERCENTAGE OF HOUSEHOLD INCOME

Estimate!!GROSS RENT AS A PERCENTAGE OF HOUSEHOLD INCOME (GRAPI)!!Occupied units paying rent (excluding units where GRAPI cannot be computed)
Estimate!!GROSS RENT AS A PERCENTAGE OF HOUSEHOLD INCOME (GRAPI)!!Occupied units paying rent (excluding units where GRAPI cannot be computed)!!Less than 15.0 percent
Estimate!!GROSS RENT AS A PERCENTAGE OF HOUSEHOLD INCOME (GRAPI)!!Occupied units paying rent (excluding units where GRAPI cannot be computed)!!15.0 to 19.9 percent
Estimate!!GROSS RENT AS A PERCENTAGE OF HOUSEHOLD INCOME (GRAPI)!!Occupied units paying rent (excluding units where GRAPI cannot be computed)!!20.0 to 24.9 percent
Estimate!!GROSS RENT AS A PERCENTAGE OF HOUSEHOLD INCOME (GRAPI)!!Occupied units paying rent (excluding units where GRAPI cannot be computed)!!25.0 to 29.9 percent
Estimate!!GROSS RENT AS A PERCENTAGE OF HOUSEHOLD INCOME (GRAPI)!!Occupied units paying rent (excluding units where GRAPI cannot be computed)!!30.0 to 3

,0,1,2,3,4,5,6,7,8,9,10
Occupied units paying rent (excluding units where GRAPI cannot be computed) counts,37418.86,28037.73,20348.44,17920.86,14318.82,17911.90,16896.39,25042.42,37602.94,21886.06,19395.59
Occupied units paying rent (excluding units where GRAPI cannot be computed) counts percent,0.15,0.11,0.08,0.07,0.06,0.07,0.07,0.10,0.15,0.09,0.08
Occupied units paying rent (excluding units where GRAPI cannot be computed) moe,1491.56,1235.67,1083.02,990.15,864.66,957.63,961.17,1097.31,1395.10,1070.15,916.36
Occupied units paying rent (excluding units where GRAPI cannot be computed) moa percent,0.02,0.03,0.03,0.03,0.04,0.03,0.03,0.03,0.02,0.03,0.03
Less than 15.0 percent counts,4975.35,2779.95,1855.17,2109.18,1123.74,2161.13,1968.13,2469.74,3873.73,2490.56,2061.32
Less than 15.0 percent counts percent,0.02,0.01,0.01,0.01,0.00,0.01,0.01,0.01,0.02,0.01,0.01
Less than 15.0 percent moe,701.90,497.41,369.51,394.52,357.98,374.96,445.78,459.40,529.20,487.01,366.20
Less than 15.0 percent moa percent,0.09,0.11,0.12,0.11,0.19,0.11,0.14,0.11,0.08,0.12,0.11
15.0 to 19.9 percent counts,4873.36,3101.47,2378.82,1919.95,1723.98,1750.43,2404.59,3180.49,5460.37,2620.44,2067.11
15.0 to 19.9 percent counts percent,0.02,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.02,0.01,0.01


In [28]:
with pd.ExcelWriter('sa2020_ccd_housing_update.xlsx') as writer:
    final1.to_excel(writer, sheet_name=groups[0][:30])
    final2.to_excel(writer, sheet_name=groups[1][:30])
    final3.to_excel(writer, sheet_name=groups[2][:30])
    final4.to_excel(writer, sheet_name=groups[3][:30])
    final5.to_excel(writer, sheet_name=groups[4][:30])
    final6.to_excel(writer, sheet_name=groups[5][:30])
    final7.to_excel(writer, sheet_name=groups[6][:30])
    final8.to_excel(writer, sheet_name=groups[7][:30])
    final9.to_excel(writer, sheet_name=groups[8][:30])
    final10.to_excel(writer, sheet_name=groups[9][:30])
    final11.to_excel(writer, sheet_name=groups[10][:30])        
    final12.to_excel(writer, sheet_name=groups[11][:30])
    final13.to_excel(writer, sheet_name=groups[12][:30])
    final14.to_excel(writer, sheet_name=groups[13][:30])
    final15.to_excel(writer, sheet_name='MONTHLY COST W MORTGAGE')
    final16.to_excel(writer, sheet_name='MONTHLY COST WO MORTGAGE')
    final17.to_excel(writer, sheet_name=groups[15][:30])
    final18.to_excel(writer, sheet_name=groups[16][:30])

In [27]:
len('MONTHLY COST W MORTGAGE')

23